<a href="https://colab.research.google.com/github/alvi2496/DeCaf/blob/master/DeCaf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notebook for the experiment of building **DeCaf** (**De**sign **C**l**a**ssi**f**ier)

### Objective
The main objective is to classify discussions from pull request, issue tracker commit messages and code comments as `design` or `general`. We also want to make the classifier cross project compatible.

### Data Collection
- We intent to have three types of data. One data is to train the `Word Embedding` model. As `Word Embedding` requires structured form of literature, we have used sentences from literatures(ex. papers and books). Also we have restricted our choice of papers and books to only from the Software Engineering domain for keep the context of our `Word Embedding` model restricted to Software Engineering. Our `Word Embedding` model will be used to vectorize our train data.
- We have collected our train data from Stack Overflow questions, answers and comments. We have collected data and classified them as `design` or `general` based on the tag. For example, questions and answers that contain `design-patterns` or `software-design` falls under the class of `design` while data tagged as `javascript` or `django` as classified as `general`

### Data Cleaning
Raw data can have a lot of noise. Specially when scraped from documents of website, it can contain a lot of misinformation in the form of names, punctuations, numbers(ex. years), misspelled and incompleted words. Also it can have a lot of stopwords that can make the model confused. We have removed all this to make our data as clean as possible. After the cleaning process, out data only contains words that are not stopwords, present in the english dictionary and has lenght greater than three.

In [24]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Assign Data location
- literature: holds the data for word embedding

In [0]:
literature_file = "/content/drive/My Drive/UVic-RA/DeCaf/data/literature.txt"
we_model_file = "/content/drive/My Drive/UVic-RA/DeCaf/model/we.bin"

### Create Word Embedding
- Use fasttext for word embedding
1. we take literature.txt as our input data.
2. we train the classifier upsupervised since we just want to group the data according to the similarity.
3. we have used skipgram as we have observe that skipgram models works better with subword information that cbow
4. we are taking words with character number from 4-20. since we are removing every word less than three characters, its not important to take the characters less than 4 characters. Also, the design words seems to be on the bigger side. for ex. reproduceability contains 16 characters. we are considering characters upto 25 characters.
5. We are taking 300 dimension of the words. Also looping for 10 epochs. Both because the training corpus is relatively small.

In [26]:
!pip install fasttext

### Train the `Word Embedding` model and save it as we.bin
- Train and save the model if the model is not present
- Load the model from disk if preselt

In [27]:
import os
from datetime import datetime as dt
import fasttext as ft

if not os.path.exists(we_model_file):
  prints(str(dt.now())+' Training Word Embedding model...')
  model = ft.train_unsupervised(literature_file, "skipgram", minn=4, maxn=25, dim=300, epoch=10)
  model.save_model(we_model_file)
  print(str(dt.now())+' Model trained and saved successfully')
else:
  print(str(dt.now())+' Loading Word Embedding model from disk...')
  model = ft.load_model(we_model_file)
  print(str(dt.now())+' Model loaded successfully.')

2020-03-16 08:59:03.262021 Loading Word Embedding model from disk...


2020-03-16 08:59:12.913266 Model loaded successfully.


### Playing around with the model

Get the dimention of the model

In [28]:
model.get_dimension()

300

Get the words of the model... For a demo... 

In [29]:
model.get_words()

['software',
 'design',
 'system',
 'data',
 'development',
 'process',
 'code',
 'model',
 'used',
 'systems',
 'test',
 'using',
 'information',
 'engineering',
 'project',
 'analysis',
 'work',
 'time',
 'when',
 'different',
 'testing',
 'will',
 'example',
 'quality',
 'approach',
 'number',
 'based',
 'case',
 'requirements',
 'security',
 'figure',
 'class',
 'method',
 'program',
 'research',
 'application',
 'results',
 'service',
 'first',
 'user',
 'models',
 'methods',
 'study',
 'developers',
 'however',
 'team',
 'knowledge',
 'level',
 'state',
 'components',
 'section',
 'services',
 'table',
 'component',
 'projects',
 'problem',
 'tools',
 'management',
 'source',
 'techniques',
 'university',
 'architecture',
 'context',
 'support',
 'agile',
 'cases',
 'object',
 'need',
 'order',
 'type',
 'language',
 'users',
 'tool',
 'implementation',
 'product',
 'computer',
 'interface',
 'applications',
 'performance',
 'change',
 'three',
 'provide',
 'specific',
 'programm

Get the vector of a word

In [0]:
model.get_word_vector('Maintainability')

Get the vector of a sentence

In [0]:
model.get_sentence_vector("Add performance tracker to active admin jobs")